# Detect Model Bias with Amazon SageMaker Clarify


## Amazon Science: _[How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)_ 

[<img src="img/amazon_science_clarify.png"  width="100%" align="left">](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

# Terminology

* **Bias**: 
An imbalance in the training data or the prediction behavior of the model across different groups, such as age or income bracket. Biases can result from the data or algorithm used to train your model. For instance, if an ML model is trained primarily on data from middle-aged individuals, it may be less accurate when making predictions involving younger and older people.

* **Bias metric**: 
A function that returns numerical values indicating the level of a potential bias.

* **Bias report**:
A collection of bias metrics for a given dataset, or a combination of a dataset and a model.

* **Label**:
Feature that is the target for training a machine learning model. Referred to as the observed label or observed outcome.

* **Positive label values**:
Label values that are favorable to a demographic group observed in a sample. In other words, designates a sample as having a positive result.

* **Negative label values**:
Label values that are unfavorable to a demographic group observed in a sample. In other words, designates a sample as having a negative result.

* **Facet**:
A column or feature that contains the attributes with respect to which bias is measured.

* **Facet value**:
The feature values of attributes that bias might favor or disfavor.

# Posttraining Bias Metrics
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html

* **Difference in Positive Proportions in Predicted Labels (DPPL)**:
Measures the difference in the proportion of positive predictions between the favored facet a and the disfavored facet d.

* **Disparate Impact (DI)**:
Measures the ratio of proportions of the predicted labels for the favored facet a and the disfavored facet d.

* **Difference in Conditional Acceptance (DCAcc)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for predicted positive outcomes (acceptances).

* **Difference in Conditional Rejection (DCR)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for negative outcomes (rejections).

* **Recall Difference (RD)**:
Compares the recall of the model for the favored and disfavored facets.

* **Difference in Acceptance Rates (DAR)**:
Measures the difference in the ratios of the observed positive outcomes (TP) to the predicted positives (TP + FP) between the favored and disfavored facets.

* **Difference in Rejection Rates (DRR)**:
Measures the difference in the ratios of the observed negative outcomes (TN) to the predicted negatives (TN + FN) between the disfavored and favored facets.

* **Accuracy Difference (AD)**:
Measures the difference between the prediction accuracy for the favored and disfavored facets.

* **Treatment Equality (TE)**:
Measures the difference in the ratio of false positives to false negatives between the favored and disfavored facets.

* **Conditional Demographic Disparity in Predicted Labels (CDDPL)**:
Measures the disparity of predicted labels between the facets as a whole, but also by subgroups.

* **Counterfactual Fliptest (FT)**:
Examines each member of facet d and assesses whether similar members of facet a have different model predictions.


In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Test data for bias

We created test data in JSONLines format to match the model inputs. 

In [ ]:
test_data_bias_path = "./data-clarify/test_data_bias.jsonl"

In [ ]:
!head -n 1 $test_data_bias_path

### Upload the data

In [ ]:
test_data_bias_s3_uri = sess.upload_data(bucket=bucket, key_prefix="bias/test_data_bias", path=test_data_bias_path)
test_data_bias_s3_uri

In [ ]:
!aws s3 ls $test_data_bias_s3_uri

In [ ]:
%store test_data_bias_s3_uri

# Run Posttraining Model Bias Analysis

In [ ]:
%store -r training_job_name

In [ ]:
try:
    training_job_name
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous TRAIN section before you continue.")
    print("+++++++++++++++++++++++++++++++")

In [ ]:
print(training_job_name)

## Create Model

In [ ]:
import sagemaker

inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.3.1",
    py_version="py37",
    instance_type="ml.m5.4xlarge",
    image_scope="inference",
)
print(inference_image_uri)

In [ ]:
model_name = sess.create_model_from_job(training_job_name=training_job_name, image_uri=inference_image_uri)
print(model_name)

# SageMakerClarifyProcessor

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.c5.2xlarge", 
    sagemaker_session=sess
)

# Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

Similarly, the `ModelConfig` object communicates information about your trained model and `ModelPredictedLabelConfig` provides information on the format of your predictions.  

**Note**: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. `ModelConfig` specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

## DataConfig

In [ ]:
bias_report_prefix = "bias/report-{}".format(training_job_name)

bias_report_output_path = "s3://{}/{}".format(bucket, bias_report_prefix)

data_config = clarify.DataConfig(
    s3_data_input_path=test_data_bias_s3_uri,
    s3_output_path=bias_report_output_path,
    label="star_rating",
    features="features",
    # label must be last, features in exact order as passed into model
    headers=["review_body", "product_category", "star_rating"],
    dataset_type="application/jsonlines",
)

## ModelConfig

In [ ]:
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.4xlarge",
    instance_count=1,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    # {"features": ["the worst", "Digital_Software"]}
    content_template='{"features":$features}',
)

## _Note: `label` is set to the JSON key for the model prediction results_

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(label="predicted_label")

## BiasConfig

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[
        5,
        4,
    ],  # needs to be int or str for continuous dtype, needs to be >1 for categorical dtype
    facet_name="product_category",
)

# Run Clarify Job

In [ ]:
clarify_processor.run_post_training_bias(
    data_config=data_config,
    data_bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    #    methods='all', # FlipTest requires all columns to be numeric
    methods=["DPPL", "DI", "DCA", "DCR", "RD", "DAR", "DRR", "AD", "TE"],
    wait=False,
    logs=False,
)

In [ ]:
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, bias_report_prefix
        )
    )
)

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=run_post_training_bias_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

# Download Report From S3

In [ ]:
!aws s3 ls $bias_report_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))

# View Bias Report in Studio
In Studio, you can view the results under the experiments tab.

<img src="img/bias_report.gif">

Each bias metric has detailed explanations with examples that you can explore.

<img src="img/bias_detail.gif">

You could also summarize the results in a handy table!

<img src="img/bias_report_chart.gif">

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>